In [1]:
!pip install ../input/bird-panns/torchlibrosa-master/torchlibrosa-master/

Processing /kaggle/input/bird-panns/torchlibrosa-master/torchlibrosa-master
  Created wheel for torchlibrosa: filename=torchlibrosa-0.0.4-py3-none-any.whl size=8864 sha256=090bfe128f55c05ce9f81cc352c9c0a4aaad4bc047536f1ac6895c840c5861a1
  Stored in directory: /root/.cache/pip/wheels/52/35/08/7a90aa926e1403318b7b36ba5a03ad940bd5002dad343b8c1f
Successfully built torchlibrosa
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import os
import gc
import time
import shutil
import random
import warnings
import typing as tp
from pathlib import Path
from contextlib import contextmanager
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
from glob import glob
from joblib import delayed, Parallel

import cv2
import librosa
import audioread
import soundfile as sf
import librosa.display 
from fastprogress import progress_bar
from scipy.io import wavfile
from librosa.core import resample, to_mono

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data.sampler import Sampler

#from efficientnet_pytorch import EfficientNet
#from apex import amp
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

import albumentations
from albumentations.core.transforms_interface import DualTransform, BasicTransform

kernel_start_time = time.perf_counter()

In [3]:
class config:
    period= 10
    num_fold = 5
    num_class = 264
    num_workers = 2
    
    ####################
    #training parameters
    ####################
    verbose = True
    verbose_step = 1
    folder = 'panns'
    use_mixup = True
    use_amp = False
    accumulate_steps = 1
    batch_size = 32
    lr = 3.5111757055874305e-05
    n_epochs = 30
    weight_decay = 0.01
    
    step_scheduler = False
    validation_scheduler = True
    SchedulerClass = torch.optim.lr_scheduler.CosineAnnealingLR
    scheduler_params = dict(
        T_max=50
    )
#     SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
#     scheduler_params = dict(
#         mode='min',
#         factor=0.5,
#         patience=1,
#         verbose=False, 
#         threshold=0.0001,
#         threshold_mode='abs',
#         cooldown=0, 
#         min_lr=1e-8,
#         eps=1e-08
#     )

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_seed()
warnings.filterwarnings("ignore")

In [5]:
ROOT = Path.cwd().parent
INPUT_ROOT = ROOT / "input"
RAW_DATA = INPUT_ROOT / "birdsong-recognition"
TRAIN_AUDIO_DIR = RAW_DATA / "train_audio"
TRAIN_RESAMPLED_AUDIO_DIRS = [
    INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
]
TEST_AUDIO_DIR = RAW_DATA / "test_audio"

train = pd.read_csv(TRAIN_RESAMPLED_AUDIO_DIRS[0] / "train_mod.csv")

if not TEST_AUDIO_DIR.exists():
    TEST_AUDIO_DIR = INPUT_ROOT / "birdcall-check" / "test_audio"
    test = pd.read_csv(INPUT_ROOT / "birdcall-check" / "test.csv")
else:
    test = pd.read_csv(RAW_DATA / "test.csv")

## Split Train/Val DataSet

In [6]:
tmp_list = []
for audio_d in TRAIN_RESAMPLED_AUDIO_DIRS:
    if not audio_d.exists():
        continue
    for ebird_d in audio_d.iterdir():
        if ebird_d.is_file():
            continue
        for wav_f in ebird_d.iterdir():
            tmp_list.append([ebird_d.name, wav_f.name, wav_f.as_posix()])
            
train_wav_path_exist = pd.DataFrame(
    tmp_list, columns=["ebird_code", "resampled_filename", "file_path"])

del tmp_list

train_all = pd.merge(
    train, train_wav_path_exist, on=["ebird_code", "resampled_filename"], how="inner")

print(train.shape)
print(train_wav_path_exist.shape)
print(train_all.shape)

(21375, 38)
(21375, 3)
(21375, 39)


In [7]:
skf = StratifiedKFold(config.num_fold)

train_all["fold"] = -1
for fold_id, (train_index, val_index) in enumerate(skf.split(train_all, train_all["ebird_code"])):
    train_all.iloc[val_index, -1] = fold_id
    
# # check the propotion
fold_proportion = pd.pivot_table(train_all, index="ebird_code", 
                                 columns="fold", values="xc_id", aggfunc=len)
print(fold_proportion.shape)

(264, 5)


In [8]:
use_fold = 0
train_file_list = train_all.query("fold != @use_fold")[["file_path", "ebird_code"]].values.tolist()
val_file_list = train_all.query("fold == @use_fold")[["file_path", "ebird_code"]].values.tolist()

print("[fold {}] train: {}, val: {}".format(use_fold, len(train_file_list), len(val_file_list)))

[fold 0] train: 17100, val: 4275


## Make Labels Dectionary

In [9]:
#https://www.kaggle.com/ttahara/training-birdsong-baseline-resnest50-fast
BIRD_CODE = {
    'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4,
    'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9,
    'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14,
    'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19,
    'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24,
    'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29,
    'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34,
    'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39,
    'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44,
    'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49,
    'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'cacwre': 54,
    'calgul': 55, 'calqua': 56, 'camwar': 57, 'cangoo': 58, 'canwar': 59,
    'canwre': 60, 'carwre': 61, 'casfin': 62, 'caster1': 63, 'casvir': 64,
    'cedwax': 65, 'chispa': 66, 'chiswi': 67, 'chswar': 68, 'chukar': 69,
    'clanut': 70, 'cliswa': 71, 'comgol': 72, 'comgra': 73, 'comloo': 74,
    'commer': 75, 'comnig': 76, 'comrav': 77, 'comred': 78, 'comter': 79,
    'comyel': 80, 'coohaw': 81, 'coshum': 82, 'cowscj1': 83, 'daejun': 84,
    'doccor': 85, 'dowwoo': 86, 'dusfly': 87, 'eargre': 88, 'easblu': 89,
    'easkin': 90, 'easmea': 91, 'easpho': 92, 'eastow': 93, 'eawpew': 94,
    'eucdov': 95, 'eursta': 96, 'evegro': 97, 'fiespa': 98, 'fiscro': 99,
    'foxspa': 100, 'gadwal': 101, 'gcrfin': 102, 'gnttow': 103, 'gnwtea': 104,
    'gockin': 105, 'gocspa': 106, 'goleag': 107, 'grbher3': 108, 'grcfly': 109,
    'greegr': 110, 'greroa': 111, 'greyel': 112, 'grhowl': 113, 'grnher': 114,
    'grtgra': 115, 'grycat': 116, 'gryfly': 117, 'haiwoo': 118, 'hamfly': 119,
    'hergul': 120, 'herthr': 121, 'hoomer': 122, 'hoowar': 123, 'horgre': 124,
    'horlar': 125, 'houfin': 126, 'houspa': 127, 'houwre': 128, 'indbun': 129,
    'juntit1': 130, 'killde': 131, 'labwoo': 132, 'larspa': 133, 'lazbun': 134,
    'leabit': 135, 'leafly': 136, 'leasan': 137, 'lecthr': 138, 'lesgol': 139,
    'lesnig': 140, 'lesyel': 141, 'lewwoo': 142, 'linspa': 143, 'lobcur': 144,
    'lobdow': 145, 'logshr': 146, 'lotduc': 147, 'louwat': 148, 'macwar': 149,
    'magwar': 150, 'mallar3': 151, 'marwre': 152, 'merlin': 153, 'moublu': 154,
    'mouchi': 155, 'moudov': 156, 'norcar': 157, 'norfli': 158, 'norhar2': 159,
    'normoc': 160, 'norpar': 161, 'norpin': 162, 'norsho': 163, 'norwat': 164,
    'nrwswa': 165, 'nutwoo': 166, 'olsfly': 167, 'orcwar': 168, 'osprey': 169,
    'ovenbi1': 170, 'palwar': 171, 'pasfly': 172, 'pecsan': 173, 'perfal': 174,
    'phaino': 175, 'pibgre': 176, 'pilwoo': 177, 'pingro': 178, 'pinjay': 179,
    'pinsis': 180, 'pinwar': 181, 'plsvir': 182, 'prawar': 183, 'purfin': 184,
    'pygnut': 185, 'rebmer': 186, 'rebnut': 187, 'rebsap': 188, 'rebwoo': 189,
    'redcro': 190, 'redhea': 191, 'reevir1': 192, 'renpha': 193, 'reshaw': 194,
    'rethaw': 195, 'rewbla': 196, 'ribgul': 197, 'rinduc': 198, 'robgro': 199,
    'rocpig': 200, 'rocwre': 201, 'rthhum': 202, 'ruckin': 203, 'rudduc': 204,
    'rufgro': 205, 'rufhum': 206, 'rusbla': 207, 'sagspa1': 208, 'sagthr': 209,
    'savspa': 210, 'saypho': 211, 'scatan': 212, 'scoori': 213, 'semplo': 214,
    'semsan': 215, 'sheowl': 216, 'shshaw': 217, 'snobun': 218, 'snogoo': 219,
    'solsan': 220, 'sonspa': 221, 'sora': 222, 'sposan': 223, 'spotow': 224,
    'stejay': 225, 'swahaw': 226, 'swaspa': 227, 'swathr': 228, 'treswa': 229,
    'truswa': 230, 'tuftit': 231, 'tunswa': 232, 'veery': 233, 'vesspa': 234,
    'vigswa': 235, 'warvir': 236, 'wesblu': 237, 'wesgre': 238, 'weskin': 239,
    'wesmea': 240, 'wessan': 241, 'westan': 242, 'wewpew': 243, 'whbnut': 244,
    'whcspa': 245, 'whfibi': 246, 'whtspa': 247, 'whtswi': 248, 'wilfly': 249,
    'wilsni1': 250, 'wiltur': 251, 'winwre3': 252, 'wlswar': 253, 'wooduc': 254,
    'wooscj2': 255, 'woothr': 256, 'y00475': 257, 'yebfly': 258, 'yebsap': 259,
    'yehbla': 260, 'yelwar': 261, 'yerwar': 262, 'yetvir': 263
}

INV_BIRD_CODE = {v: k for k, v in BIRD_CODE.items()}

In [10]:
ebird_codes =  train_all.query("fold != @use_fold")["ebird_code"].values.tolist()
all_targets = []
for i in range(len(ebird_codes)):
    ebird_code = ebird_codes[i]
    labels = np.zeros(len(BIRD_CODE), dtype="f")
    labels[BIRD_CODE[ebird_code]] = 1
    all_targets.append(labels)
    
all_targets = np.array(all_targets)

## Define DataLoader

In [11]:
class SimpleBalanceClassSampler(Sampler):

    def __init__(self, targets, classes_num):

        self.targets = targets
        self.classes_num = classes_num
        
        self.samples_num_per_class = np.sum(self.targets, axis=0)
        self.max_num = np.max(self.samples_num_per_class)
        
        self.indexes_per_class = []
        # Training indexes of all sound classes. E.g.: 
        # [[0, 11, 12, ...], [3, 4, 15, 16, ...], [7, 8, ...], ...]
        for k in range(self.classes_num):
            self.indexes_per_class.append(
                np.where(self.targets[:, k] == 1)[0])
        
        self.length = self.classes_num * self.max_num

    def __iter__(self):
        
        all_indexs = []
        
        for k in range(self.classes_num):
            if len(self.indexes_per_class[k]) == self.max_num:
                all_indexs.append(self.indexes_per_class[k])
            else:
                gap = self.max_num - len(self.indexes_per_class[k])
                random_choice = np.random.choice(self.indexes_per_class[k], int(gap), replace=True)
                all_indexs.append(np.array(list(random_choice) + list(self.indexes_per_class[k])))
                
        l = np.stack(all_indexs).T
        l = l.reshape(-1)
        random.shuffle(l)
        return iter(l)

    def __len__(self):
        return int(self.length)

## Augmentations

In [12]:
def calculate_rms(samples):
    """Given a numpy array of audio samples, return its Root Mean Square (RMS)."""
    return np.sqrt(np.mean(np.square(samples), axis=-1))


def calculate_desired_noise_rms(clean_rms, snr):
    """
    Given the Root Mean Square (RMS) of a clean sound and a desired signal-to-noise ratio (SNR),
    calculate the desired RMS of a noise sound to be mixed in.
    Based on https://github.com/Sato-Kunihiko/audio-SNR/blob/8d2c933b6c0afe6f1203251f4877e7a1068a6130/create_mixed_audio_file.py#L20
    :param clean_rms: Root Mean Square (RMS) - a value between 0.0 and 1.0
    :param snr: Signal-to-Noise (SNR) Ratio in dB - typically somewhere between -20 and 60
    :return:
    """
    a = float(snr) / 20
    noise_rms = clean_rms / (10 ** a)
    return noise_rms

In [13]:
class AudioTransform(BasicTransform):
    """Transform for Audio task"""

    @property
    def targets(self):
        return {"data": self.apply}
    
    def update_params(self, params, **kwargs):
        if hasattr(self, "interpolation"):
            params["interpolation"] = self.interpolation
        if hasattr(self, "fill_value"):
            params["fill_value"] = self.fill_value
        return params

    
class NoiseInjection(AudioTransform):
    """It simply add some random value into data by using numpy"""
    def __init__(self, always_apply=False, p=0.5):
        super(NoiseInjection, self).__init__(always_apply, p)
    
    def apply(self, data, noise_levels=(0, 0.05), **params):
        sound, sr = data
        noise_level = np.random.uniform(*noise_levels)
        noise = np.random.randn(len(sound))
        augmented_sound = sound + noise_level * noise
        # Cast back to same data type
        augmented_sound = augmented_sound.astype(type(sound[0]))

        return augmented_sound, sr

    
class AddGaussianSNR(AudioTransform):
    """It simply add some random value into data by using numpy"""
    def __init__(self, min_SNR=0.001, max_SNR=1.0, always_apply=False, p=0.5):
        super(AddGaussianSNR, self).__init__(always_apply, p)
        self.min_SNR = min_SNR
        self.max_SNR = max_SNR
    
    def apply(self, data, **params):
        sound, sr = data
        std = np.std(sound)
        noise_std = random.uniform(self.min_SNR * std, self.max_SNR * std)
        noise = np.random.normal(0.0, noise_std, size=len(sound)).astype(np.float32)
        return sound + noise, sr


class ShiftingTime(AudioTransform):
    """Shifting time axis"""
    def __init__(self, always_apply=False, p=0.5):
        super(ShiftingTime, self).__init__(always_apply, p)
    
    def apply(self, data, **params):
        sound, sr = data

        shift_max = np.random.randint((len(sound)/sr)/2)
        shift = np.random.randint(sr * shift_max)
        direction = np.random.randint(0,2)
        if direction == 1:
            shift = -shift

        augmented_sound = np.roll(sound, shift)
        # Set to silence for heading/ tailing
        if shift > 0:
            augmented_sound[:shift] = 0
        else:
            augmented_sound[shift:] = 0

        return augmented_sound, sr

    
class Pitch_Speed(AudioTransform):
    """Shifting time axis"""
    def __init__(self, always_apply=False, p=0.5):
        super(Pitch_Speed, self).__init__(always_apply, p)   
    
    def apply(self, data, **params):
        
        sound, sr = data
        
        length_change = np.random.uniform(low=0.8, high = 1)
        speed_fac = 1.0  / length_change
    
        tmp = np.interp(np.arange(0,len(sound),speed_fac),np.arange(0,len(sound)),sound)
        minlen = min(sound.shape[0], tmp.shape[0])
        sound *= 0
        sound[0:minlen] = tmp[0:minlen]
        
        return sound, sr      
        
        
class RandomAudio(AudioTransform):
    """Shifting time axis"""
    def __init__(self,  seconds=10, always_apply=False, p=0.5):
        super(RandomAudio, self).__init__(always_apply, p)

        self.seconds = seconds
    
    def apply(self, data, **params):
        sound, sr = data

        shift = np.random.randint(len(sound))
        trim_sound = np.roll(sound, shift)

        min_samples = int(sr * self.seconds)

        if len(trim_sound) < min_samples:
            padding = min_samples - len(trim_sound)
            offset = padding // 2
            trim_sound = np.pad(trim_sound, (offset, padding - offset), "constant")
        else:
            trim_sound = trim_sound[:min_samples]

        return trim_sound, sr
    
class PitchShift(AudioTransform):
    """Shifting time axis"""
    def __init__(self, always_apply=False, p=0.5):
        super(PitchShift, self).__init__(always_apply, p)
    
    def apply(self, data, **params):
        sound, sr = data

        n_steps = np.random.randint(-10, 10)
        augmented_sound = librosa.effects.pitch_shift(sound, sr, n_steps)

        return augmented_sound, sr

class Gain(AudioTransform):
    """
    Multiply the audio by a random amplitude factor to reduce or increase the volume. This
    technique can help a model become somewhat invariant to the overall gain of the input audio.
    """

    def __init__(self, min_gain_in_db=-12, max_gain_in_db=12, always_apply=False,p=0.5):
        super(Gain,self).__init__(always_apply,p)
        assert min_gain_in_db <= max_gain_in_db
        self.min_gain_in_db = min_gain_in_db
        self.max_gain_in_db = max_gain_in_db


    def apply(self, data, **args):
        sound, sr = data
        amplitude_ratio = 10**(random.uniform(self.min_gain_in_db, self.max_gain_in_db)/20)
        return sound * amplitude_ratio, sr

class PolarityInversion(AudioTransform):
    def __init__(self, always_apply=False, p=0.5 ):
        super(PolarityInversion, self).__init__(always_apply, p)
        
    def apply(self,data,**params):
        '''
        data : ndarray of audio timeseries
        '''
        sound, sr = data
        return -sound, sr


class CutOut(AudioTransform):
    def __init__(self, always_apply=False, p=0.5 ):
        super(CutOut, self).__init__(always_apply, p)
        
    def apply(self,data,**params):
        '''
        data : ndarray of audio timeseries
        '''
        sound, sr = data
        start_ = np.random.randint(0,len(sound))
        end_ = np.random.randint(start_,len(sound))
        
        sound[start_:end_] = 0
        
        return sound, sr

class AddCustomNoise(AudioTransform):
    """
    This Function allows you to add noise from any custom file you want.
    The idea is to load the sounds files first as a list of arrays using 
    librosa so that at inference time we do not have to wait for that.
    """
    def __init__(self, sound_array, always_apply=False, p=0.5 ):
        super(AddCustomNoise, self).__init__(always_apply, p)
        '''
        The array must be of [sound1, sound2, sound3, ....]
        '''
        
        self.sound_array = sound_array
        
    def apply(self,data,**params):
        '''
        data : ndarray of audio timeseries
        ''' 
        sound, sr = data
        noise = self.sound_array[int(np.random.uniform(0,len(self.sound_array)))]
        
        
        if len(noise)>len(sound):
            start_ = len(noise)-len(sound)
            noise = noise[start_ : start_+len(sound)] 
        else:
            noise = np.pad(noise, (0, len(sound)-len(noise)), "constant")
        
        start_ = np.random.randint(0,len(sound))
        end_ = np.random.randint(start_,len(sound))
            
        sound[start_:end_] = noise[start_:end_]

        return sound, sr

class AddBackgroundNoise(AudioTransform):
    """Adds background noise and does mix-up of two audio files."""
    def __init__(self, sound_array, min_snr_in_db=3, max_snr_in_db=30, always_apply=False, p=0.5 ):
        super(AddBackgroundNoise, self).__init__(always_apply, p)
        '''
        The array must be of [sound1, sound2, sound3, ....]
        '''
        
        self.sound_array = sound_array
        self.min_snr_in_db = min_snr_in_db
        self.max_snr_in_db = max_snr_in_db
    
    def apply(self, data, **params):
        sound, sr = data
        noise = self.sound_array[int(np.random.uniform(0,len(self.sound_array)))]
        
        snr_in_db = random.uniform(self.min_snr_in_db, self.max_snr_in_db)
        
        clean_rms = calculate_rms(sound)
        noise_rms = calculate_rms(noise)
        desired_noise_rms = calculate_desired_noise_rms(
            clean_rms, snr_in_db
        )

        # Adjust the noise to match the desired noise RMS
        noise = noise * (desired_noise_rms / noise_rms)
        
        # Repeat the sound if it shorter than the input sound
        num_samples = len(sound)
        while len(noise) < num_samples:
            noise = np.concatenate((noise, noise))

        if len(noise) > num_samples:
            noise = noise[0:num_samples]
        
        return sound + noise, sr

In [14]:
noise_files = os.listdir('../input/freesound-audio-tagging/audio_test/')
noise_files = noise_files[0:50]

noise_sound_array = []
for f in noise_files:
    wave, sr = librosa.load(os.path.join('../input/freesound-audio-tagging/audio_test/', f))
    noise_sound_array.append(wave)

In [15]:
audio_augs = albumentations.Compose([
    RandomAudio(seconds=config.period,always_apply=True),
    AddGaussianSNR(p=0.6),
    NoiseInjection(p=0.1),
    Pitch_Speed(p=0.3),
    Gain(p=0.6),
    PolarityInversion(p=0.3),
    AddBackgroundNoise(noise_sound_array, p=0.6),
    albumentations.OneOf([
        CutOut(p=0.5),
        AddCustomNoise(noise_sound_array, p=0.7)  
    ])
#     PitchShift(p=0.2),  ## too long
#     TimeStretch(p=0.3),
])

### Small Test of Augs

In [16]:
import IPython.display as ipd
audio_path = '../input/birdsong-recognition/train_audio/aldfly/XC134874.mp3'

y,sr = librosa.load(audio_path,sr=32500)

print('Audio Intially')
ipd.Audio(y, rate=sr)

Audio Intially


In [17]:
new_y, new_sr = audio_augs(data=(y, sr))['data']
ipd.Audio(new_y,rate=sr)

In [18]:
class birddataset(data.Dataset):
    """
    Based On
    https://www.kaggle.com/ttahara/training-birdsong-baseline-resnest50-fast/data?
    """
    
    def __init__(self, file_list, test=False, label_smooth=True, wave_transforms = None):
        
        self.file_list = file_list
        self.label_smooth = label_smooth
        self.test = test
        self.wave_transforms = wave_transforms
 
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx: int):
        
        wav_path, ebird_code = self.file_list[idx]
        wave, sr = sf.read(wav_path)
        effective_length = sr * config.period
        
        if self.wave_transforms is not None:
            if len(wave) < effective_length:
                new_wave = np.zeros(effective_length, dtype=wave.dtype)
                start = np.random.randint(effective_length - len(wave))
                new_wave[start:start + len(wave)] = wave
                wave = new_wave.astype(np.float32)
            elif len(wave) > effective_length:
                start = np.random.randint(len(wave) - effective_length)
                wave = wave[start:start + effective_length].astype(np.float32)
            else:
                wave= wave.astype(np.float32)
            
            wave, sr = self.wave_transforms(data=(wave, sr))['data']
            wave = wave.astype(np.float32)
        else:
            if len(wave) < effective_length:
                new_wave = np.zeros(effective_length, dtype=wave.dtype)
                start = np.random.randint(effective_length - len(wave))
                new_wave[start:start + len(wave)] = wave
                wave = new_wave.astype(np.float32)
            elif len(wave) > effective_length:
                start = np.random.randint(len(wave) - effective_length)
                wave = wave[start:start + effective_length].astype(np.float32)
            else:
                wave= wave.astype(np.float32)
        
        labels = np.zeros(len(BIRD_CODE), dtype="f")
        labels[BIRD_CODE[ebird_code]] = 1
        if self.label_smooth and self.test == False:
            labels = self.make_label_smooth(labels, num_classes=config.num_class)
        
        return {"waveform": wave, "targets": labels}
    
    def make_label_smooth(self, labels, num_classes, epsilon=0.1):
        b = np.ones(num_classes) * (1 / num_classes)
        return (1 - epsilon) * labels + epsilon * b 

## Define Model

In [19]:
class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return np.array(mixup_lambdas)

In [20]:
def do_mixup(x, mixup_lambda):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes 
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0 :: 2].transpose(0, -1) * mixup_lambda[0 :: 2] + \
        x[1 :: 2].transpose(0, -1) * mixup_lambda[1 :: 2]).transpose(0, -1)
    return out

In [21]:
def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)

In [22]:
def _resnet_conv3x3(in_planes, out_planes):
    #3x3 convolution with padding
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1,
                     padding=1, groups=1, bias=False, dilation=1)


def _resnet_conv1x1(in_planes, out_planes):
    #1x1 convolution
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, bias=False)

In [23]:
class _ResnetBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(_ResnetBasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('_ResnetBasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in _ResnetBasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1

        self.stride = stride

        self.conv1 = _resnet_conv3x3(inplanes, planes)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = _resnet_conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

        self.init_weights()

    def init_weights(self):
        init_layer(self.conv1)
        init_bn(self.bn1)
        init_layer(self.conv2)
        init_bn(self.bn2)
        nn.init.constant_(self.bn2.weight, 0)

    def forward(self, x):
        identity = x

        if self.stride == 2:
            out = F.avg_pool2d(x, kernel_size=(2, 2))
        else:
            out = x

        out = self.conv1(out)
        out = self.bn1(out)
        out = self.relu(out)
        out = F.dropout(out, p=0.1, training=self.training)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)

        return out

In [24]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x

In [25]:
class _ResNet(nn.Module):
    def __init__(self, block, layers, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(_ResNet, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group

        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            if stride == 1:
                downsample = nn.Sequential(
                    _resnet_conv1x1(self.inplanes, planes * block.expansion),
                    norm_layer(planes * block.expansion),
                )
                init_layer(downsample[0])
                init_bn(downsample[1])
            elif stride == 2:
                downsample = nn.Sequential(
                    nn.AvgPool2d(kernel_size=2), 
                    _resnet_conv1x1(self.inplanes, planes * block.expansion),
                    norm_layer(planes * block.expansion),
                )
                init_layer(downsample[1])
                init_bn(downsample[2])

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        return x

In [26]:
class ResNet38(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num):
        
        super(ResNet38, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        # self.conv_block2 = ConvBlock(in_channels=64, out_channels=64)

        self.resnet = _ResNet(block=_ResnetBasicBlock, layers=[3, 4, 6, 3], zero_init_residual=True)

        self.conv_block_after1 = ConvBlock(in_channels=512, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)

        self.init_weights()

    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)


    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = self.resnet(x)
        x = F.avg_pool2d(x, kernel_size=(2, 2))
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = self.conv_block_after1(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        embedding = F.dropout(x, p=0.5, training=self.training)
        clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return output_dict

In [27]:
class ConvPreWavBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvPreWavBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=3, stride=1,
                              padding=1, bias=False)
                              
        self.conv2 = nn.Conv1d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=3, stride=1, dilation=2, 
                              padding=2, bias=False)
                              
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        x = F.max_pool1d(x, kernel_size=pool_size)
        
        return x
    
class Wavegram_Logmel_Cnn14(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num):
        
        super(Wavegram_Logmel_Cnn14, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None

        self.pre_conv0 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=11, stride=5, padding=5, bias=False)
        self.pre_bn0 = nn.BatchNorm1d(64)
        self.pre_block1 = ConvPreWavBlock(64, 64)
        self.pre_block2 = ConvPreWavBlock(64, 128)
        self.pre_block3 = ConvPreWavBlock(128, 128)
        self.pre_block4 = ConvBlock(in_channels=4, out_channels=64)

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=128, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_layer(self.pre_conv0)
        init_bn(self.pre_bn0)
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)
 
    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        # Wavegram
        a1 = F.relu_(self.pre_bn0(self.pre_conv0(input[:, None, :])))
        a1 = self.pre_block1(a1, pool_size=4)
        a1 = self.pre_block2(a1, pool_size=4)
        a1 = self.pre_block3(a1, pool_size=4)
        a1 = a1.reshape((a1.shape[0], -1, 32, a1.shape[-1])).transpose(2, 3)
        a1 = self.pre_block4(a1, pool_size=(2, 1))

        # Log mel spectrogram
        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
            a1 = do_mixup(a1, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')

        # Concatenate Wavegram and Log mel spectrogram along the channel dimension
        x = torch.cat((x, a1), dim=1)

        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        embedding = F.dropout(x, p=0.5, training=self.training)
        clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return output_dict

In [28]:
def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.

    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)
        
class PANNsCNN14Att(nn.Module):
    
    def __init__(self, sample_rate: int, window_size: int, hop_size: int,
                 mel_bins: int, fmin: int, fmax: int, classes_num: int):
        super().__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.interpolate_ratio = 32  # Downsampled ratio

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(
            n_fft=window_size,
            hop_length=hop_size,
            win_length=window_size,
            window=window,
            center=center,
            pad_mode=pad_mode,
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(
            sr=sample_rate,
            n_fft=window_size,
            n_mels=mel_bins,
            fmin=fmin,
            fmax=fmax,
            ref=ref,
            amin=amin,
            top_db=top_db,
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(
            time_drop_width=64,
            time_stripes_num=2,
            freq_drop_width=8,
            freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(mel_bins)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.att_block = AttBlock(2048, classes_num, activation='sigmoid')

        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        
    def cnn_feature_extractor(self, x):
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        return x
    
    def preprocess(self, input, mixup_lambda=None):
        # t1 = time.time()
        x = self.spectrogram_extractor(input)  # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)  # (batch_size, 1, time_steps, mel_bins)

        frames_num = x.shape[2]

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        return x, frames_num
        

    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""
        x, frames_num = self.preprocess(input, mixup_lambda=mixup_lambda)

        # Output shape (batch size, channels, time, frequency)
        x = self.cnn_feature_extractor(x)
        
        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)

        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        output_dict = {
            'framewise_output': framewise_output,
            'clipwise_output': clipwise_output
        }

        return output_dict

In [29]:
def get_model(num_classes, pretrained=False, weight_path=None):
    
    model_config = {
        "sample_rate": 32000,
        "window_size": 1024,
        "hop_size": 320,
        "mel_bins": 64,
        "fmin": 50,
        "fmax": 14000,
        }

    model_config["classes_num"] = 527
    model = ResNet38(**model_config)
    model.fc_audioset = nn.Linear(2048, num_classes, bias=True)
    init_layer(model.fc_audioset)
    weights = torch.load("../input/resnet-38-fold-0-50-epochs/panns/best-acc-checkpoint.bin")
    model.load_state_dict(weights['model_state_dict'])   
    return model

## Define Trainer

In [30]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [31]:
def clip_bce(output_dict, target_dict):
    """Binary crossentropy loss.
    """
    return F.binary_cross_entropy(
        output_dict['clipwise_output'], target_dict)

In [32]:
#https://www.kaggle.com/hidehisaarai1213/introduction-to-sound-event-detection
class PANNsLoss(nn.Module):
    def __init__(self):
        super().__init__()

        self.bce = nn.BCELoss()

    def forward(self, input, target):
        input_ = input["clipwise_output"]
        input_ = torch.where(torch.isnan(input_),
                             torch.zeros_like(input_),
                             input_)

        input_ = torch.clamp(input_, 0.0, 1.0)
        target = target.float()

        return self.bce(input_, target)

In [33]:
class Fitter:
    
    def __init__(self, model, device, config, start_time=kernel_start_time):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5
        self.best_score = 0.0
        self.early_stop_count = 0
        
        self.model = model
        self.device = device
        self.use_amp = config.use_amp
        self.accumulate_steps = config.accumulate_steps
        self.model.to(self.device)
        self.use_mixup = config.use_mixup
        self.threshold = 0.5
        
        self.start_time = start_time
        
        if self.use_mixup:
            self.mixup_augmenter = Mixup(mixup_alpha=1.)
        self.loss_fn = PANNsLoss()
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
        
        if self.use_amp:
            self.model, self.optimizer = amp.initialize(self.model, self.optimizer, opt_level="O1",verbosity=0)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        
        best_epoch = 0
        for e in range(self.config.n_epochs):
            if time.perf_counter() - self.start_time > 31600:
                print("Time limit reached! Stopping kernel!"); break
            else: print("Continue training...")
            if self.early_stop_count == 5:
                self.log(f'early stopping: Epoch: {self.epoch}')
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')

            t = time.time()
            summary_loss, f1_score, acc_score = self.validation(self.model, validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, val_f1_score: {f1_score:.5f}, val_acc_score: {acc_score:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
                    os.remove(path)
            else:
                self.early_stop_count += 1
            if f1_score > self.best_score:
                self.best_score = f1_score
                self.model.eval()
                self.save(f'{self.base_dir}/best-acc-checkpoint.bin')

            if self.config.validation_scheduler:
                self.scheduler.step()
                #self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1
            
            
    def validation(self, model, val_loader):
        model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        pred_results  = []
        origin_labels = []
        
        for step, batch_data_dict in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                batch_size = batch_data_dict['waveform'].shape[0]
                batch_output_dict = self.model(torch.FloatTensor(batch_data_dict['waveform']).cuda(), None)
                batch_target_dict = {'target': torch.FloatTensor(batch_data_dict['targets']).cuda()}
                loss = clip_bce(batch_output_dict, batch_target_dict['target'])
                
                proba = batch_output_dict['clipwise_output'].detach().cpu().numpy()
                y_pred = proba.argmax(axis=1)
                y_true = batch_data_dict['targets'].argmax(axis=1)
                
                pred_results.append(y_pred)
                origin_labels.append(y_true)
                
                summary_loss.update(loss.detach().item(), batch_size)
            
        pred_results = np.concatenate(pred_results)    
        origin_labels = np.concatenate(origin_labels)
        val_f1_score = f1_score(origin_labels, pred_results, average='macro')
        val_accuracy_score = accuracy_score(origin_labels, pred_results)

        return summary_loss, val_f1_score, val_accuracy_score

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        self.optimizer.zero_grad() #very important
        for step, batch_data_dict in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            batch_size = batch_data_dict['waveform'].shape[0]
            
            if self.use_mixup:
                batch_data_dict['mixup_lambda'] = self.mixup_augmenter.get_lambda(len(batch_data_dict['waveform']))
                batch_size = batch_size//2
                batch_output_dict = self.model(torch.FloatTensor(batch_data_dict['waveform']).cuda(), 
                                               torch.FloatTensor(batch_data_dict['mixup_lambda']).cuda())
                batch_target_dict = {'target': torch.FloatTensor(do_mixup(batch_data_dict['targets'], 
                                                                          batch_data_dict['mixup_lambda'][:, np.newaxis])).cuda()}
            else:
                batch_output_dict = self.model(torch.FloatTensor(batch_data_dict['waveform']).cuda(), None)
                batch_target_dict = {'target': torch.FloatTensor(batch_data_dict['targets']).cuda()}
            
            loss = clip_bce(batch_output_dict, batch_target_dict['target'])
            if self.use_amp:
                with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            if (step+1) % self.accumulate_steps == 0: # Wait for several backward steps
                self.optimizer.step()                 # Now we can do an optimizer step
                self.optimizer.zero_grad()

            summary_loss.update(loss.detach().item(), batch_size)

            if self.config.step_scheduler:
                self.scheduler.step()
                #self.scheduler.step(metrics=summary_loss.avg)

            
        return summary_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({'model_state_dict': self.model.state_dict()}, path)
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [34]:
def collate_fn(list_data_dict):
    """Collate data.
    Args:
      list_data_dict, e.g., [{'waveform': (clip_samples,), ...}, 
                             {'waveform': (clip_samples,), ...},
                             ...]
    Returns:
      np_data_dict, dict, e.g.,
          {'waveform': (batch_size, clip_samples), ...}
    """
    np_data_dict = {}
    
    for key in list_data_dict[0].keys():
        #[print(data_dict[key].shape) for data_dict in list_data_dict]
        np_data_dict[key] = np.array([data_dict[key] for data_dict in list_data_dict])
    
    return np_data_dict

In [35]:
def run_training(net, config):
    device = torch.device('cuda:0')
    
    train_dataset = birddataset(train_file_list, label_smooth=True, test=False, wave_transforms=audio_augs)
    validation_dataset = birddataset(val_file_list, label_smooth=False, test=True, wave_transforms=None)
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.batch_size, #*2 if config.use_mixup else config.batch_size,
        sampler=SimpleBalanceClassSampler(all_targets, config.num_class),
        pin_memory=False,
        drop_last=True,
        collate_fn=collate_fn,
        num_workers=config.num_workers,
    )
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=config.batch_size,
        num_workers=config.num_workers,
        shuffle=False,
        collate_fn = collate_fn,
        pin_memory=False,
    )

    fitter = Fitter(model=net, device=device, config=config)
    fitter.fit(train_loader, val_loader)

## Start Training

In [36]:
net = get_model(num_classes=config.num_class, pretrained=True)
run_training(net, config)

Fitter prepared. Device is cuda:0
Continue training...

2020-09-10T21:53:50.837970
LR: 3.5111757055874305e-05
[RESULT]: Train. Epoch: 0, summary_loss: 0.01437, time: 806.93440
[RESULT]: Val. Epoch: 0, summary_loss: 0.00812, val_f1_score: 0.70408, val_acc_score: 0.72421, time: 111.94183
Continue training...

2020-09-10T22:09:11.158766
LR: 3.507711453985841e-05
[RESULT]: Train. Epoch: 1, summary_loss: 0.01431, time: 754.65566
[RESULT]: Val. Epoch: 1, summary_loss: 0.00823, val_f1_score: 0.70845, val_acc_score: 0.72421, time: 104.55382
Continue training...

2020-09-10T22:23:31.841644
LR: 3.4973323709994776e-05
[RESULT]: Train. Epoch: 2, summary_loss: 0.01423, time: 746.29897
[RESULT]: Val. Epoch: 2, summary_loss: 0.00833, val_f1_score: 0.70316, val_acc_score: 0.71813, time: 105.71131
Continue training...

2020-09-10T22:37:44.467476
LR: 3.480079418127136e-05
[RESULT]: Train. Epoch: 3, summary_loss: 0.01421, time: 667.80503
[RESULT]: Val. Epoch: 3, summary_loss: 0.00808, val_f1_score: 0.709

### Kaggle Dataset

In [37]:
# !pip install --upgrade --force-reinstall --no-deps kaggle

In [38]:
# import zipfile
# import shutil

# os.environ['KAGGLE_USERNAME'] =  ""
# os.environ['KAGGLE_KEY'] =  ""

In [39]:
# dataset_path = "./panns/"
# !ls -ls "$dataset_path"
# ! kaggle datasets init -p "$dataset_path"
# !cat "$dataset_path"/dataset-metadata.json

In [40]:
# import json
# metadata = {}
# metadata['licenses'] = [{'name': 'CC0-1.0'}]
# metadata['id'] = "urvishp80/panns10SecModelAdamwRP"
# metadata['title'] = "10secSEDmodelNewAugsAdamwRP"
# with open(dataset_path+"/dataset-metadata.json","w") as mfile:
#     json.dump(metadata, mfile)

In [41]:
# !cat "$dataset_path"/dataset-metadata.json
# !kaggle datasets create -p "$dataset_path"

## OOF(Simple)

In [42]:
INV_BIRD_CODE = {v: k for k, v in BIRD_CODE.items()}
val_true = train_all.query("fold == @use_fold")["ebird_code"].values.tolist()

In [43]:
def row_wise_f1_score_micro(y_true, y_pred):
    """ author @shonenkov """
    F1 = []
    for preds, trues in zip(y_pred, y_true):
        TP, FN, FP = 0, 0, 0
        preds = preds.split()
        trues = trues.split()
        for true in trues:
            if true in preds:
                TP += 1
            else:
                FN += 1
        for pred in preds:
            if pred not in trues:
                FP += 1
        F1.append(2*TP / (2*TP + FN + FP))
    return np.mean(F1)

In [44]:
def get_trained_model(weight_path=None):
    
    model_config = {
        "sample_rate": 32000,
        "window_size": 1024,
        "hop_size": 320,
        "mel_bins": 64,
        "fmin": 50,
        "fmax": 14000,
        "classes_num":264
        }

    model = ResNet38(**model_config)
    if weight_path:
        print("load pretrain weight: {}".format(weight_path))
        weights = torch.load(weight_path, map_location=torch.device('cpu'))
        model.load_state_dict(weights['model_state_dict'])
    model.cuda()
    model.eval()
    
    return model

best_weight_path = glob(f'{config.folder}/best-checkpoint-*epoch.bin')[-1]
trained_net = get_trained_model(best_weight_path)

load pretrain weight: panns/best-checkpoint-028epoch.bin


In [45]:
class TestDataset(data.Dataset):
    def __init__(self, file_list):
        
        self.file_list = file_list 
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx: int):
        
        SR = 32000
        wav_path, ebird_code = self.file_list[idx]
        y, sr = sf.read(wav_path)
        effective_length = sr * config.period
        
        if len(y) > effective_length:
            y = y.astype(np.float32)
            len_y = len(y)
            start = 0
            end = SR * 5
            y_all = []
            while len_y > start:
                y_batch = y[start:end].astype(np.float32)
                if len(y_batch) != (SR * 5):
                    y_pad = np.zeros(5 * SR, dtype=np.float32)
                    y_pad[:len(y_batch)] = y_batch
                    y_all.append(y_pad)
                    break
                start = end
                end = end + SR * 5
                y_all.append(y_batch)
            y_all = np.asarray(y_all)
            return y_all
        elif len(y) < effective_length:
            new_wave = np.zeros(effective_length, dtype=y.dtype)
            start = np.random.randint(effective_length - len(y))
            new_wave[start:start + len(y)] = y
            y = new_wave.astype(np.float32)
        else:
            y = y.astype(np.float32)

        return y

In [46]:
def prediction(val_file_list, model, threshold):
    
    SR = 32000
    period = config.period
    dataset = TestDataset(val_file_list)
    loader = data.DataLoader(dataset, batch_size=1, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.eval()
    #j = 0
    predictions = []
    for wave in progress_bar(loader):

        if wave.size()[1] == SR*period: 
            wave = wave.to(device).float()

            with torch.no_grad():
                prediction = model(wave)
                event_proba = prediction['clipwise_output'].detach().cpu().numpy().reshape(-1)
                
            events = proba >= threshold
            labels = np.argwhere(events).reshape(-1).tolist()

        else:
            # to avoid prediction on large batch
            wave = wave.squeeze(0)
            batch_size = 16
            whole_size = wave.size(0)
            if whole_size % batch_size == 0:
                n_iter = whole_size // batch_size
            else:
                n_iter = whole_size // batch_size + 1
                
            all_events = set()
            for batch_i in range(n_iter):
                batch = wave[batch_i * batch_size:(batch_i + 1) * batch_size]
                if batch.ndim == 3:
                    batch = batch.unsqueeze(0)

                batch = batch.to(device)
                with torch.no_grad():
                    prediction = model(batch)
                    proba = prediction['clipwise_output'].detach().cpu().numpy()
                    
                events = proba >= threshold
                for i in range(len(events)):
                    event = events[i, :]
                    labels = np.argwhere(event).reshape(-1).tolist()
                    for label in labels:
                        all_events.add(label)
                        
            labels = list(all_events)
        if len(labels) == 0:
            predictions.append("nocall")
        else:
            labels_str_list = list(map(lambda x: INV_BIRD_CODE[x], labels))
            label_string = " ".join(labels_str_list)
            predictions.append(label_string)

    return predictions

In [47]:
def threshold_optimization(val_file_list, val_true, model):
    
    best_score = 0.0
    best_threshold = 0.0
    for threshold in np.arange(0.1, 1.0, 0.1):
        val_predict = prediction(val_file_list, model, threshold)
        score = row_wise_f1_score_micro(val_true, val_predict)
        
        if score >= best_score:
            best_score = score
            best_threshold = threshold

    print(best_score, best_threshold)

In [48]:
# threshold_optimization(val_file_list, val_true, model=trained_net)